## ✈️ 국내 항공편의 지역을 예측할 수 있는 모델 구축
### ✅ 목표
- 2018년 국내선 도착 지연에 가장 큰 영향을 미친 요인을 파악한다.
- 2019년에는 항공사로 인한 지연을 이런 요인 중 일부를 완화하기에 충분한 정확도로 예측한다.
### 📚 사전지식
- OTP(정시운항률)는 항공기 운영 시간 엄수에 대해 널리 인정받는 지표이다.
- OTP는 예정된 도착 시간의 15분 이내에 도착한 항공편의 비율로 측정된다.
### 🚘 접근법
1. 다양한 회귀방법론으로 지연 시간 예측
2. 다양한 분류 방법론으로 항공편의 지연 여부 분류
3. 차원 축소 방법론으로 지연된 항공편 시각화

In [17]:
import pandas as pd
import math
import numpy as np
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import PolynomialFeatures, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn import metrics, linear_model, tree, naive_bayes, neighbors, ensemble, neural_network, svm, decomposition, manifold
from rulefit import RuleFit
import statsmodels.api as sm
from interpret.glassbox import ExplainableBoostingClassifier
from interpret import show
from interpret.perf import ROC


In [8]:
aad18_df = pd.read_csv('2018.csv').drop(columns=['FL_DATE', 'OP_CARRIER'])
aad18_df.head()

,OP_CARRIER_FL_NUM,ORIGIN,DEST,CRS_DEP_TIME,DEP_TIME,DEP_DELAY,TAXI_OUT,WHEELS_OFF,WHEELS_ON,TAXI_IN,...,CRS_ELAPSED_TIME,ACTUAL_ELAPSED_TIME,AIR_TIME,DISTANCE,CARRIER_DELAY,WEATHER_DELAY,NAS_DELAY,SECURITY_DELAY,LATE_AIRCRAFT_DELAY,Unnamed: 27
0,2429,EWR,DEN,1517,1512.0,-5.0,15.0,1527.0,1712.0,10.0,...,268.0,250.0,225.0,1605.0,NaN,NaN,NaN,NaN,NaN,NaN
1,2427,LAS,SFO,1115,1107.0,-8.0,11.0,1118.0,1223.0,7.0,...,99.0,83.0,65.0,414.0,NaN,NaN,NaN,NaN,NaN,NaN
2,2426,SNA,DEN,1335,1330.0,-5.0,15.0,1345.0,1631.0,5.0,...,134.0,126.0,106.0,846.0,NaN,NaN,NaN,NaN,NaN,NaN
3,2425,RSW,ORD,1546,1552.0,6.0,19.0,1611.0,1748.0,6.0,...,190.0,182.0,157.0,1120.0,NaN,NaN,NaN,NaN,NaN,NaN
4,2424,ORD,ALB,630,650.0,20.0,13.0,703.0,926.0,10.0,...,112.0,106.0,83.0,723.0,NaN,NaN,NaN,NaN,NaN,NaN


In [10]:
aad18_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7213446 entries, 0 to 7213445
Data columns (total 26 columns):
 #   Column               Dtype  
---  ------               -----  
 0   OP_CARRIER_FL_NUM    int64  
 1   ORIGIN               object 
 2   DEST                 object 
 3   CRS_DEP_TIME         int64  
 4   DEP_TIME             float64
 5   DEP_DELAY            float64
 6   TAXI_OUT             float64
 7   WHEELS_OFF           float64
 8   WHEELS_ON            float64
 9   TAXI_IN              float64
 10  CRS_ARR_TIME         int64  
 11  ARR_TIME             float64
 12  ARR_DELAY            float64
 13  CANCELLED            float64
 14  CANCELLATION_CODE    object 
 15  DIVERTED             float64
 16  CRS_ELAPSED_TIME     float64
 17  ACTUAL_ELAPSED_TIME  float64
 18  AIR_TIME             float64
 19  DISTANCE             float64
 20  CARRIER_DELAY        float64
 21  WEATHER_DELAY        float64
 22  NAS_DELAY            float64
 23  SECURITY_DELAY       float64
 24